# User Item Biases
* See the corresponding file in `../TrainingAlphas` for more details

In [1]:
source = "ExplicitUserItemBiases";

In [2]:
import NBInclude: @nbinclude
@nbinclude("Alpha.ipynb");
@nbinclude("../TrainingAlphas/Explicit/ExplicitUserItemBiasesBase.ipynb");

In [3]:
function train_model(training, λ, a, stop_criteria)
    λ_u, _, λ_wu, λ_wa = λ
    @info "training model with parameters [$λ_u, $λ_wu, $λ_wa]"
    users, items, ratings = training.user, training.item, training.rating
    weights =
        expdecay(length(training.item), log(λ_wu)) .*
        expdecay(get_counts("training", false; by_item = true), log(λ_wa))
    u = zeros(eltype(λ_u), maximum(users))
    ρ_u = zeros(eltype(u), length(u), Threads.nthreads())
    Ω_u = zeros(eltype(u), length(u), Threads.nthreads())
    while !stop!(stop_criteria, [u])
        update_users!(users, items, ratings, weights, u, a, λ_u, ρ_u, Ω_u)
        @info u
    end
    u
end;

In [4]:
function compute_alpha()
    training = get_recommendee_split(false)
    params = read_params(source)
    stop_criteria = convergence_stopper(1e-6, max_iters = 16)
    λ = params["λ"]
    u = train_model(training, λ, params["a"], stop_criteria)
    preds = make_prediction(fill(1, num_items()), collect(1:num_items()), u, params["a"])
    write_recommendee_alpha(preds, source)
end;

In [5]:
compute_alpha();

[ Info: 20220624 08:50:33 training model with parameters [0.785946, 0.6796609, 0.97671974]
Progress: 100%|███████████████████████████| Time: 0:00:00 (49.99 ns/it)39m
[ Info: 20220624 08:50:36 Float32[5.460746]
[ Info: 20220624 08:50:36 Float32[5.5915976]
[ Info: 20220624 08:50:36 Float32[5.594733]
[ Info: 20220624 08:50:36 Float32[5.594808]
[ Info: 20220624 08:50:36 Float32[5.59481]
[ Info: 20220624 08:50:36 Float32[5.59481]
